In [94]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [95]:
print(f"Tensorflow version      : {tf.__version__}")
print(f"Pandas version          : {pd.__version__}")
print(f"Numpy version           : {np.__version__}")

Tensorflow version      : 2.8.0
Pandas version          : 1.4.1
Numpy version           : 1.22.2


In [96]:
csvFile_path = "../Dataset/pedigree dataset.csv"
collumnName = ["Base Color Male", "With White Male", "Base Color Female", "With White Female", "Color Prediction"]

In [97]:
trainset = pd.read_csv(csvFile_path, names=collumnName, delimiter=",", header=0)

trainset.head(12)

,Base Color Male,With White Male,Base Color Female,With White Female,Color Prediction
0,black,0,black,0,black
1,black,0,black,1,black
2,black,0,black,1,black
3,black,1,black,1,black
4,grey,0,grey,0,grey
5,grey,1,grey,0,grey
6,grey,0,grey,1,grey
7,grey,1,grey,1,grey
8,orange,0,orange,0,orange
9,orange,1,orange,0,orange


In [98]:
trainset_feature = trainset.copy()
trainset_labels = trainset_feature.pop('Color Prediction')

In [99]:
trainset_feature.head()

,Base Color Male,With White Male,Base Color Female,With White Female
0,black,0,black,0
1,black,0,black,1
2,black,0,black,1
3,black,1,black,1
4,grey,0,grey,0


In [100]:
trainset_labels.head()

0     black
1     black
2     black
3     black
4      grey
Name: Color Prediction, dtype: object

In [135]:
def set_symbolic(dataset_feature):
    inputs = {}
    
    for name, collumn in dataset_feature.items():
        dataType = collumn.dtype
        
        if dataType == object:
            dataType = tf.string
        else:
            dataType = tf.float32
        
        inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dataType)
    
    return inputs

In [136]:
inputs = set_symbolic(trainset_feature)

In [137]:
print(inputs)

{'Base Color Male': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Base Color Male')>, 'With White Male': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'With White Male')>, 'Base Color Female': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Base Color Female')>, 'With White Female': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'With White Female')>}


In [138]:
numerics_inputs = {name:input
                   for name, input in inputs.items()
                   if input.dtype == tf.float32}

In [139]:
print(numerics_inputs.keys())

dict_keys(['With White Male', 'With White Female'])


In [140]:
x = tf.keras.layers.Concatenate()(list(numerics_inputs.values()))
norm = tf.keras.layers.Normalization()
norm.adapt(np.array(trainset[numerics_inputs.keys()]))
all_numeric_inputs = norm(x)

In [141]:
preprocessed_inputs = [all_numeric_inputs]

In [142]:
for name, input in inputs.items():
    if input.dtype == tf.float32:
        continue
    
    lookup = tf.keras.layers.StringLookup(vocabulary=np.unique(trainset_feature[name]))
    one_hot = tf.keras.layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())
    
    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x)

In [143]:
preprocessed_inputs_cat = tf.keras.layers.Concatenate()(preprocessed_inputs)
trainset_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)
tf.keras.utils.plot_model(model=trainset_preprocessing, rankdir="LR", dpi=72, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [144]:
trainset_feature_dict = {name: np.array(value)
                         for name, value, in trainset_feature.items()}

In [145]:
features_dict = {name: values[:1]
                 for name, values in trainset_feature_dict.items()}
print(trainset_preprocessing(features_dict))

tf.Tensor(
[[-0.9692234 -1.0317539  0.         1.         0.         0.
   0.         0.         1.         0.         0.         0.       ]], shape=(1, 12), dtype=float32)


In [158]:
body = tf.keras.Sequential([
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
    # tf.keras.layers.Dense(1)
    ])

preprocessed_inputs_modelBuild = trainset_preprocessing(inputs)
result = body(preprocessed_inputs_modelBuild)
model = tf.keras.Model(inputs, result)

model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Base Color Female (InputLayer)  [(None, 1)]         0           []                               
                                                                                                  
 Base Color Male (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 With White Female (InputLayer)  [(None, 1)]         0           []                               
                                                                                                  
 With White Male (InputLayer)   [(None, 1)]          0           []                               
                                                                                           

In [165]:
model.compile(
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.optimizers.Adam()
)

In [166]:
model.fit(
    x=trainset_feature_dict,
    y=trainset_labels,
    epochs=10
)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/Cast' defined at (most recent call last):
    File "c:\Users\Bugi\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Bugi\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Bugi\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\Bugi\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\Bugi\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Bugi\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", li